In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.tree import DecisionTreeClassifier
import itertools
import xgboost as xgb
from sklearn import metrics
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/malicious-urls-dataset/malicious_phish.csv


In [2]:
df = pd.read_csv("/kaggle/input/malicious-urls-dataset/malicious_phish.csv")

In [3]:
df.shape

(651191, 2)

In [4]:
length_of_url = [] # length of url
number_of_letters = [] # number of alphanumeric characters
number_of_digits = [] # number of digits
count_of_dotcom = [] # count of '.com'
count_of_codot = [] # count of '.co.'
count_of_dotnet = [] # count of '.net'
count_of_forward_slash = [] # count of '/'
count_of_percentage = [] # count of '%'
count_of_upper_case = [] # count of upper case characters
count_of_lower_case = [] # count of upper case characters
count_of_dot = [] # count of "."
count_of_upper_case = [] # count of upper case characters
count_of_lower_case = [] # count of lower case characters
count_of_dot_info = [] # count of '.info'
count_of_https = [] # count of 'https'
count_of_www_dot = [] # count of 'www.'
count_of_not_alphanumeric = [] # count of non-alphanumeric characters

for item in df['url']:
    try:
        length_of_url.append(len(item))
    except:
        length_of_url.append(0)
        
    try:
        number_of_letters.append(sum(c.isalpha() for c in item))
    except:
        number_of_letters.append(0)
        
    try:
        number_of_digits.append(sum(c.isdigit() for c in item))
    except:
        number_of_digits.append(0)
    
    try:
        count_of_dotcom.append(item.count(".com"))
    except:
        count_of_dotcom.append(0)
        
    try:
        count_of_codot.append(item.count(".co."))
    except:
        count_of_codot.append(0)
        
    try:
        count_of_dotnet.append(item.count(".net"))
    except:
        count_of_dotnet.append(0)
        
    try:
        count_of_forward_slash.append(item.count("/"))
    except:
        count_of_forward_slash.append(0)
        
    try:
        count_of_percentage.append(item.count("%"))
    except:
        count_of_percentage.append(0)

    try:
        count_of_dot.append(item.count("."))
    except:
        count_of_dot.append(0)    
        
    try:
        count_of_upper_case.append(sum(c.isupper() for c in item))
    except:
        count_of_upper_case.append(0)
    
    try:
        count_of_lower_case.append(sum(c.islower() for c in item))
    except:
        count_of_lower_case.append(0)
        
    try:
        count_of_dot_info.append(item.count(".info"))
    except:
        count_of_dot_info.append(0)
        
    try:
        count_of_https.append(item.count("https"))
    except:
        count_of_https.append(0)
        
    try:
        count_of_www_dot.append(item.count("www."))
    except:
        count_of_www_dot.append(0)
        
    try:
        count_of_not_alphanumeric.append(sum(not c.isalnum() for c in item))
    except:
        count_of_not_alphanumeric.append(0)

In [5]:
df['length_of_url'] = length_of_url
df['number_of_letters'] = number_of_letters
df['number_of_digits'] = number_of_digits
df['count_of_dotcom'] = count_of_dotcom
df['count_of_codot'] = count_of_codot
df['count_of_dotnet'] = count_of_dotnet
df['count_of_forward_slash'] = count_of_forward_slash
df['count_of_upper_case'] = count_of_upper_case
df['count_of_lower_case'] = count_of_lower_case
df['count_of_dot'] = count_of_dot
df['count_of_upper_case'] = count_of_upper_case
df['count_of_lower_case'] = count_of_lower_case
df['count_of_dot_info'] = count_of_dot_info
df['count_of_https'] = count_of_https
df['count_of_www_dot'] = count_of_www_dot
df['count_of_not_alphanumeric'] = count_of_not_alphanumeric
df['count_of_percentage'] = count_of_percentage

## Amount of symbols to letters ratio
df['not_alphanumeric_to_letters_ratio'] = df['count_of_not_alphanumeric']/df['number_of_letters']

## Amount of '%' to length ratio
df['percentage_to_length_ratio'] = df['count_of_percentage']/df['length_of_url']

## Amount of '%' to length ratio
df['percentage_to_length_ratio'] = df['count_of_percentage']/df['length_of_url']

## Amount of '/' to length ratio
df['forwards_slash_to_length_ratio'] = df['count_of_forward_slash']/df['length_of_url']

## Amount captialised vs. non-capitalised
df['upper_case_to_lower_case_ratio'] = df['count_of_upper_case']/df['count_of_lower_case']

In [6]:
df.head()

,url,type,length_of_url,number_of_letters,number_of_digits,count_of_dotcom,count_of_codot,count_of_dotnet,count_of_forward_slash,count_of_upper_case,...,count_of_dot,count_of_dot_info,count_of_https,count_of_www_dot,count_of_not_alphanumeric,count_of_percentage,not_alphanumeric_to_letters_ratio,percentage_to_length_ratio,forwards_slash_to_length_ratio,upper_case_to_lower_case_ratio
0,br-icloud.com.br,phishing,16,13,0,1,0,0,0,0,...,2,0,0,0,3,0,0.230769,0.0,0.000000,0.000000
1,mp3raid.com/music/krizz_kaliko.html,benign,35,29,1,1,0,0,2,0,...,2,0,0,0,5,0,0.172414,0.0,0.057143,0.000000
2,bopsecrets.org/rexroth/cr/1.htm,benign,31,25,1,0,0,0,3,0,...,2,0,0,0,5,0,0.200000,0.0,0.096774,0.000000
3,http://www.garage-pirenne.be/index.php?option=...,defacement,88,63,7,0,0,0,3,0,...,3,0,0,1,18,0,0.285714,0.0,0.034091,0.000000
4,http://adventure-nicaragua.net/index.php?optio...,defacement,235,199,22,0,0,1,3,63,...,2,0,0,0,14,0,0.070352,0.0,0.012766,0.463235


In [7]:
# Define X and y
X = df[['length_of_url', 'number_of_letters', 'number_of_digits',
       'count_of_dotcom', 'count_of_codot', 'count_of_dotnet',
       'count_of_forward_slash', 'count_of_upper_case', 'count_of_lower_case',
       'count_of_dot', 'count_of_dot_info', 'count_of_https',
       'count_of_www_dot', 'count_of_not_alphanumeric', 'count_of_percentage',
       'percentage_to_length_ratio','forwards_slash_to_length_ratio']]

y = df['type']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size=0.20,
    random_state=234)

print('Training dataset shape:', X_train.shape, y_train.shape)
print('Testing dataset shape:', X_test.shape, y_test.shape)

Training dataset shape: (520952, 17) (520952,)
Testing dataset shape: (130239, 17) (130239,)


In [8]:
X

,length_of_url,number_of_letters,number_of_digits,count_of_dotcom,count_of_codot,count_of_dotnet,count_of_forward_slash,count_of_upper_case,count_of_lower_case,count_of_dot,count_of_dot_info,count_of_https,count_of_www_dot,count_of_not_alphanumeric,count_of_percentage,percentage_to_length_ratio,forwards_slash_to_length_ratio
0,16,13,0,1,0,0,0,0,13,2,0,0,0,3,0,0.0,0.000000
1,35,29,1,1,0,0,2,0,29,2,0,0,0,5,0,0.0,0.057143
2,31,25,1,0,0,0,3,0,25,2,0,0,0,5,0,0.0,0.096774
3,88,63,7,0,0,0,3,0,63,3,0,0,1,18,0,0.0,0.034091
4,235,199,22,0,0,1,3,63,136,2,0,0,0,14,0,0.0,0.012766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651186,39,21,12,1,0,0,3,0,21,3,0,0,0,6,0,0.0,0.076923
651187,44,29,7,1,0,0,4,2,27,2,0,0,0,8,0,0.0,0.090909
651188,42,33,3,1,0,0,4,0,33,2,0,0,1,6,0,0.0,0.095238
651189,45,36,0,0,0,0,2,2,34,2,0,0,0,9,0,0.0,0.044444


In [ ]:
depth_list = []
accuracy_list = []

for depth in range(1,len(X.columns)):
    decision_tree = DecisionTreeClassifier(max_depth=depth)
    decision_tree.fit(X_train, y_train)
    accuracy = decision_tree.score(X_test,y_test)
    print('Depth: ',depth, ' Accuracy: ', accuracy)
    accuracy_list.append(decision_tree.score(X_test,y_test))
    depth_list.append(depth)

Depth:  1  Accuracy:  0.7361773355139397
Depth:  2  Accuracy:  0.8014726771550764
Depth:  3  Accuracy:  0.8199310498391419
Depth:  4  Accuracy:  0.8323543639002142
Depth:  5  Accuracy:  0.8408618002288101
Depth:  6  Accuracy:  0.8482482205790892
Depth:  7  Accuracy:  0.8604719016577216
Depth:  8  Accuracy:  0.8654013006856626
Depth:  9  Accuracy:  0.8701848140725896
Depth:  10  Accuracy:  0.8779090748546903
Depth:  11  Accuracy:  0.8866161441657261
Depth:  12  Accuracy:  0.8945170033553698


In [ ]:
tree_df = pd.DataFrame({'depth': depth_list, 'accuracy' : accuracy_list})

In [ ]:
import matplotlib.pyplot as plt
plt.plot(tree_df['depth'], tree_df['accuracy'])
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

decision_tree = DecisionTreeClassifier(max_depth=17)

cv_score = cross_val_score(decision_tree, X_train, y_train, cv=5)

In [ ]:
cv_score

In [ ]:
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    X,y,
    test_size=0.20,
    random_state=567)

decision_tree = DecisionTreeClassifier(max_depth=17)
decision_tree.fit(X_train, y_train)
accuracy = decision_tree.score(X_test,y_test)
y_hat = decision_tree.predict(X_test)
cm = confusion_matrix(y_test, y_hat)
print(cm)
print(sum(np.diag(cm))/sum(sum(cm))*100, '% correctly classified')

In [ ]:
array = cm
df_cm = pd.DataFrame(array, index = [i for i in ['benign', 'defacement','phishing','malware']],
                  columns = [i for i in ['benign', 'defacement','phishing','malware']])
sn.set(font_scale=1.4) # for label size
plt.figure(figsize = (10,7))
plt.ticklabel_format(style='plain', axis='y')
sn.heatmap(df_cm, annot=True, annot_kws={"size": 16}, cmap='Blues', fmt='g') # font size

In [ ]:
model = xgb.XGBClassifier(n_estimators= 1000)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

In [ ]:
cm2 = confusion_matrix(y_test, y_pred)
print(cm2)
print(sum(np.diag(cm2))/sum(sum(cm2))*100, '% correctly classified')

In [ ]:
array2 = cm2
df_cm2 = pd.DataFrame(array2, index = [i for i in ['benign', 'defacement','phishing','malware']],
                  columns = [i for i in ['benign', 'defacement','phishing','malware']])
sn.set(font_scale=1.4) # for label size
plt.figure(figsize = (10,7))
plt.ticklabel_format(style='plain', axis='y')
sn.heatmap(df_cm2, annot=True, annot_kws={"size": 16}, cmap='Blues', fmt='g') # font size